In [1]:
import pandas as pd
import numpy as np

In [3]:
from rbm_torch.utils import data_prep as dp
from rbm_torch.utils import utils
from rbm_torch.utils import seq_utils as su
from rbm_torch.analysis import analysis_methods as am

#### Starting From our Processed Fasta Files

In [8]:
last_5_rounds = ["r12", "r13", "r14", "r15", "r16"]
thc_pd = am.fetch_data(last_5_rounds, dir="./", threads=6, molecule="rna", normalize_counts=True, normalization_denominator=1e6)

In [6]:
thc_pd.head()

,sequence,copy_num,round,assignment
0,CAUAUAGGACGACGACGGUCAACACCCGCGACAACCACAGCA-,2816.857140,r12,N/A
1,CGUACCACGAGUGGGGAUUGUGGGCAGUUGGACACGCCAGUC-,1443.547078,r12,N/A
2,GCGGGAGGGGGGGAGGGGCCCGUAACAAGAACGACAUGUAAG-,795.057367,r12,N/A
3,GCUGCUCCGCAUGAGUCAUAUCGAACGAAGGGGUGUAUACGC-,643.358240,r12,N/A
4,ACACCACGGGGAGAACACCACCAGAAACACCGCCAGACUCCA-,590.439940,r12,N/A


In [9]:
thc_ct = dp.copynum_topology_faster(thc_pd, last_5_rounds, threads_per_task=2)  # Uses Total of 10 threads

Process Time 16624.104041576385


In [11]:
thc_ct.to_csv("./thc_12_to_16_ct.csv", index=False)

In [12]:
thc_ct.index.__len__()

437749